# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

experiment_name = 'arvc-azureml-project3-experiment-automl'

experiment=Experiment(ws, experiment_name)

In [19]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_cluster_name = 'arvccompute_automl_cluster'

try:
    compute_target = ComputeTarget(workspace=ws, name = compute_cluster_name)
    print('Found the compute cluster')

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_DS12_v2", max_nodes=5)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

ComputeTargetException: ComputeTargetException:
	Message: Received bad response from Resource Provider:
Response Code: 400
Headers: {'Cache-Control': 'no-cache', 'Pragma': 'no-cache', 'Content-Length': '375', 'Content-Type': 'application/json; charset=utf-8', 'Expires': '-1', 'x-ms-correlation-request-id': 'a6743d1a-d1b4-4b7e-9163-77c606b39917', 'x-ms-ratelimit-remaining-subscription-writes': '1199', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'x-ms-response-type': 'standard', 'x-ms-request-id': '|00-f40c1872748f0f45a97b7de299f1b41c-a34be01fa363434e-01.d2ebde4b_', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-Content-Type-Options': 'nosniff', 'x-request-time': '0.038', 'x-ms-routing-request-id': 'SOUTHCENTRALUS:20210318T153043Z:a6743d1a-d1b4-4b7e-9163-77c606b39917', 'Date': 'Thu, 18 Mar 2021 15:30:43 GMT'}
Content: b'{"error":{"code":"BadArgument","message":"Compute name is invalid. It can include letters, digits and dashes. It must 

In [16]:
from azureml.core.dataset import Dataset
from sklearn.model_selection import train_test_split

key = 'heart_failure_dataset'
if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print("existing dataset found.")

else:
    print("existing dataset not found. creating new one.")
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
    dataset = Dataset.Tabular.from_delimited_files(url)
    dataset = dataset.register(ws,key)

df = dataset.to_pandas_dataframe()
train, test = train_test_split(df, shuffle=True)
train.to_csv('train.csv',index = False)

datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train.csv'])

train = Dataset.Tabular.from_delimited_files([(datastore,'train.csv')])

existing dataset found.
Uploading an estimated of 1 files
Target already exists. Skipping upload for train.csv
Uploaded 0 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [17]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
automl_settings = {"primary_metric":"accuracy", "experiment_timeout_minutes":10, "enable_early_stopping":True, "n_cross_validations":5,"max_concurrent_iterations": 5}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target = compute_target, task = 'classification', training_data = train, label_column_name = 'DEATH_EVENT',**automl_settings)

In [18]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

ConfigException: ConfigException:
	Message: Compute not in 'Succeeded' state. Please choose another compute or wait till it is ready.
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Compute not in 'Succeeded' state. Please choose another compute or wait till it is ready.",
        "target": "arvccompute",
        "inner_error": {
            "code": "NotReady",
            "inner_error": {
                "code": "ComputeNotReady"
            }
        }
    }
}

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, model = remote_run.get_output()
print(best_run)
print(model)
best_run.get_tags()
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

In [ ]:
best_run_metrics

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
#TODO: Save the best model
from azureml.core import Model

# Register model
best_run.register_model(model_path='outputs/model.pkl', model_name='automl_model',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

In [10]:
from azureml.automl.core.shared import constants
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

In [ ]:
# create inference_config
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script="scoring.py",environment=myenv)

In [ ]:
from azureml.core.webservice import Webservice, AciWebservice
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [15]:
#deploy the model
service=Model.deploy(workspace=ws,
                    name="heart-deploy-service",
                    models=[model],
                    inference_config=inference_config,
                    deployment_config=deployment_config)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, print the logs of the web service and delete the service

In [16]:
print(service.get_logs())

2021-01-19T00:50:57,776027517+00:00 - rsyslog/run 
2021-01-19T00:50:57,776144220+00:00 - gunicorn/run 
2021-01-19T00:50:57,776145420+00:00 - iot-server/run 
2021-01-19T00:50:57,827470300+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

In [17]:
scoring_uri = service.scoring_uri

print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')
print(f'swagger URI: \n{service.swagger_uri}\n')

print(service.scoring_uri)
print(service.swagger_uri)


service state: Healthy

scoring URI: 
http://ec7be877-6f29-47ac-aab9-cbdf6f72b95f.southcentralus.azurecontainer.io/score

swagger URI: 
http://ec7be877-6f29-47ac-aab9-cbdf6f72b95f.southcentralus.azurecontainer.io/swagger.json

http://ec7be877-6f29-47ac-aab9-cbdf6f72b95f.southcentralus.azurecontainer.io/score
http://ec7be877-6f29-47ac-aab9-cbdf6f72b95f.southcentralus.azurecontainer.io/swagger.json


In [18]:
import requests
import json

data={"data":
  [{'age': 87.0,
  'anaemia': 0,
  'creatinine_phosphokinase': 981,
  'diabetes': 0,
  'ejection_fraction': 55,
  'high_blood_pressure': 1,
  'platelets': 368000,
  'serum_creatinine': 3.5,
  'serum_sodium': 137,
  'sex': 0,
  'smoking': 0,
  'time': 4}]
  }

input_data=json.dumps(data)

with open("data.json","w") as file:
    file.write(input_data)

headers={"Content-Type":"application/json"}

result=requests.post(scoring_uri, input_data, headers=headers)
print("The output is: ",result.json())

The output is:  Must pass 2-d input


In [19]:
result.status_code

200

In [20]:
service.delete()
compute_target.delete()
model.delete()
run.delete()
experiment.delete()

NameError: name 'run' is not defined